In [109]:
using Plots
using LaTeXStrings
include("MetrologicalModel.jl")
include("HiddenMarkovModel.jl")

GenerateStringHMM (generic function with 1 method)

In [119]:
# Simulation parameters.
θ = 0.3
string_length = 100
number_trajectories = 20
initial_state_HMM = "A"
initial_state_metrological = "D"

"D"

In [120]:
# Create and setup the metrological model for the modified SNS model.
model = CreateMetrologicalModel("Model")
AddMetrologicalState(model, "E")
AddMetrologicalState(model, "D")
AddEdge(model, "E", "D", "0", 1-θ, -1/(1-θ)^2)
AddEdge(model, "E", "D", "1", θ, - 1/θ^2)
AddEdge(model, "D", "D", "0", 1, 0)
AddEdge(model, "D", "E", "1", 1, 0)
Initialise(model, "D")

Dict{String, Any} with 5 entries:
  "MetrologicalStates" => Any["E", "D"]
  "CurrentStateIndex"  => 2
  "Name"               => "Model"
  "CurrentStateName"   => "D"
  "Edges"              => Any[Dict{String, Any}("FinalStateIndex"=>2, "Emission…

In [121]:
# Create and setup the HMM for the modified SNS model.
HMM = CreateHMM("HMM")
AddStateHMM(HMM, "A")
AddStateHMM(HMM, "B")
AddStateHMM(HMM, "C")
AddEdgeHMM(HMM, "A", "A", "0", 0.5)
AddEdgeHMM(HMM, "A", "B", "0", 0.5)
AddEdgeHMM(HMM, "B", "B", "0", 1/3.)
AddEdgeHMM(HMM, "B", "A", "1", 2/3 * θ)
AddEdgeHMM(HMM, "B", "C", "1", 2/3 * (1 - θ))
AddEdgeHMM(HMM, "C", "A", "1", 1)
InitialiseHMM(HMM, "A")

Dict{String, Any} with 5 entries:
  "States"            => Any["A", "B", "C"]
  "CurrentStateIndex" => 1
  "Name"              => "HMM"
  "CurrentStateName"  => "A"
  "Edges"             => Any[Dict{String, Any}("FinalStateIndex"=>1, "Emission"…

In [122]:
GenerateStringHMM(HMM, string_length)

"0110110110000011001100110011001101000011001100010110110110001000000011011010110010110000110110110001"

In [123]:
v_fisher = zeros(string_length)
v_v_fisher_trajectories = []

for trajectory in 1:number_trajectories
    # Generate string.
    Initialise(model, initial_state_metrological)
    InitialiseHMM(HMM, initial_state_HMM)
    
    v_fisher_trajectory = []
    fisher_trajectory = 0
    for item in 1:string_length
        emission, HMM = EvolveHMM(HMM)
        update, fisher_contribution, fumm = EvolveAndComputeFisherContribution(model, emission)

        fisher_trajectory -= fisher_contribution
        v_fisher[item] += fisher_trajectory / number_trajectories
        push!(v_fisher_trajectory, fisher_trajectory)
    end
    push!(v_v_fisher_trajectories, v_fisher_trajectory)
end

In [125]:
plot(v_fisher, linewidth=3, legend=nothing, xlabel=L"$N$", ylabel=L"$FI$")
plot!(v_v_fisher_trajectories, alpha=.1)
savefig("Fisher_information_trajectories.pdf")